In [1]:
import pandas as pd 
import os

In [44]:
# make dirs 

xnat_root = '/nfs/masi/MCL/xnat'
maintain_root = '/nfs/masi/MCL/QA/maintain_list'
xnat_list = os.listdir(xnat_root)
for xnat_name in xnat_list:
    xname = xnat_name.split('_')[0]
    if xname[:4] == 'xnat' and len(xname[4:]) == 8 and xname[4:] > '20200120':
        if not os.path.exists(maintain_root + '/' + xname[4:]):
            os.mkdir(maintain_root + '/' + xname[4:])
        print (xname)
    
    #print (xname)
print ()

In [139]:
# got the QA_bad from slicedir
f = open('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/slicesdir.txt')
lines = f.readlines()

lines = [line.split(' ')[0] for line in lines]
f = open('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/QA_bad.txt', 'w')
for sess in lines:
    f.write(sess + '\n')
f.close()


In [142]:
# generate QA_good from QA_bad
data_root = '/nfs/masi/MCL/xnat/xnat20200928_makeupjustin'
df = pd.read_csv(data_root + '/dicomQA_step1.csv')
all_list = df['sess'].tolist() 

bad_list = open(data_root + '/QA_bad.txt').readlines()
bad_list = [i.strip() for i in bad_list]
print ("this should be None", set(bad_list) - set(all_list)) 

uncertain_list = open(data_root + '/lookgoodInstanceNumberbad.txt').readlines()
uncertain_list = [i.strip() for i in uncertain_list]
print ("this should be None", set(uncertain_list) - set(all_list)) 

good_list = set(all_list) - set(bad_list) - set(uncertain_list)
f = open(data_root + '/QA_good.txt', 'w')
for item in good_list:
    f.write(item + '\n')
f.close()

this should be None set()
this should be None set()


In [164]:
# the past_root and current_root should be near

# NOt uncertain verison

past_root = '/nfs/masi/MCL/QA/maintain_list/20200928'
current_root = '/nfs/masi/MCL/QA/maintain_list/20201002'

past_bad = pd.read_csv(past_root + '/uptonow_bad.csv')['sess'].tolist()

past_good = pd.read_csv(past_root + '/uptonow_good.csv')['sess'].tolist()

current_bad = open(current_root + '/QA_bad.txt').readlines()
current_bad = [i.strip().split(' ')[0] for i in current_bad]

current_good = open(current_root + '/QA_good.txt').readlines()
current_good = [i.strip().split(' ')[0] for i in current_good]

total_bad = set(current_bad) | (set(past_bad) - set(current_good))

total_good = set(current_good) | (set(past_good) - set(current_bad))

data = pd.DataFrame()

data['sess'] = list(total_good)

data.to_csv(current_root + '/uptonow_good.csv', index = False)

data = pd.DataFrame()

data['sess'] = list(total_bad)

data.to_csv(current_root + '/uptonow_bad.csv', index = False)

print (len(past_bad), len(past_good), len(total_bad), len(total_good))


117 4953 113 5018


In [118]:
# find the looks good, but with not good instance number
f = open('/nfs/masi/MCL/xnat/xnat20201002_mightallbad/sldicesdirerr.txt')
dirlines = f.readlines()
lines = [i.strip().split(' ')[0] for i in dirlines]
df = pd.read_csv('/nfs/masi/MCL/xnat/xnat20201002_mightallbad/dicomQA.csv')
dfbad_list = df.loc[df['auto'] == 'bad']['sess'].tolist()


In [122]:
f = open('/nfs/masi/MCL/xnat/xnat20201002_mightallbad/lookbadInstanceNumbergood.txt', 'w')
for item in set(lines) - set(dfbad_list):
    f.write(item + '\n')
f.close()


In [121]:
f = open('/nfs/masi/MCL/xnat/xnat20201002_mightallbad/lookgoodInstanceNumberbad.txt', 'w')
for item in set(dfbad_list) - set(lines):
    f.write(item + '\n')
f.close()


21 4953 50 5018


In [174]:
past_bad = pd.read_csv(past_root + '/uptonow_bad.csv')['sess'].tolist()
past_uncer = pd.read_csv(past_root + '/uptonow_uncertain.csv')['sess'].tolist()
new_bad = list(set(past_bad) - set(past_uncer))
data = pd.DataFrame()
data['sess'] = new_bad
data.to_csv(past_root + '/uptonow_bad.csv', index = False)

In [178]:
# the past_root and current_root should be near

# with uncertain verison
# begin at 20200928

past_root = '/nfs/masi/MCL/QA/maintain_list/20201002'
current_root = '/nfs/masi/MCL/QA/maintain_list/20201003'

past_bad = pd.read_csv(past_root + '/uptonow_bad.csv')['sess'].tolist()
past_uncer = pd.read_csv(past_root + '/uptonow_uncertain.csv')['sess'].tolist()
past_good = pd.read_csv(past_root + '/uptonow_good.csv')['sess'].tolist()


current_bad = open(current_root + '/QA_bad.txt').readlines()
current_bad = [i.strip().split(' ')[0] for i in current_bad]

current_good = open(current_root + '/QA_good.txt').readlines()
current_good = [i.strip().split(' ')[0] for i in current_good]

current_uncer = open(current_root + '/lookgoodInstanceNumberbad.txt').readlines()
current_uncer = [i.strip().split(' ')[0] for i in current_uncer]

assert (len(set(past_good) & set(current_bad)) == 0) 
assert (len(set(past_good) & set(current_uncer)) == 0) 
assert (len(set(past_uncer) & set(current_bad)) == 0)

total_bad = set(current_bad) | (set(past_bad) - set(current_good) - set(current_uncer))

total_uncer = set(current_uncer) | (set(past_uncer) - set(current_good))

total_good = set(current_good) | (set(past_good) - set(current_bad) - set(current_good))

print ('The following 4 should be all None')
print (total_bad & total_uncer, total_good & total_uncer, total_good & total_bad)
print (set(past_bad) & set(past_uncer))

data = pd.DataFrame()

data['sess'] = list(total_good)

data.to_csv(current_root + '/uptonow_good.csv', index = False)

data = pd.DataFrame()

data['sess'] = list(total_bad)

data.to_csv(current_root + '/uptonow_bad.csv', index = False)

data = pd.DataFrame()

data['sess'] = list(total_uncer)

data.to_csv(current_root + '/uptonow_uncertain.csv', index = False)

print (len(past_bad), len(past_good), len(past_uncer), len(total_bad), len(total_good), len(total_uncer))



The following 4 should be all None
set() set() set()
set()
66 5018 50 68 5045 50


In [160]:
data_root = '/nfs/masi/MCL/QA/maintain_list/20200928'
good_list = open(data_root + '/QA_good.txt').readlines()
good_list = [i.strip() for i in good_list]
bad_list = open(data_root + '/QA_bad.txt').readlines()
bad_list = [i.strip() for i in bad_list]
uncer_list = open(data_root + '/lookgoodInstanceNumberbad.txt').readlines()
uncer_list = [i.strip() for i in uncer_list]
print (set(good_list) & set(bad_list))
print (set(uncer_list) & set(bad_list))
print (set(good_list) & set(uncer_list))
print (len(set(good_list)), len(set(bad_list)), len(set(uncer_list)))
good_list2 = pd.read_csv(data_root + '/uptonow_good.csv')['sess'].tolist()
print (set(good_list2) & set(bad_list))

set()
{'41292354607-20150123', '26685639639-20150527'}
set()
217 10 21
set()


In [167]:
# ensure not overlapp
data_root = '/nfs/masi/MCL/QA/maintain_list/20201002'
good_list = pd.read_csv(data_root + '/uptonow_good.csv')['sess'].tolist()
bad_list = pd.read_csv(data_root + '/uptonow_bad.csv')['sess'].tolist()
uncer_list = pd.read_csv(data_root + '/uptonow_uncertain.csv')['sess'].tolist()
print (set(good_list) & set(bad_list))
print (set(uncer_list) & set(bad_list))
print (set(good_list) & set(uncer_list))
print (len(set(good_list)), len(set(bad_list)), len(set(uncer_list)))

{'9937003500-20140327', '10817759785_20140905_155950_512', '20981799122_20130409_090122', '18123771639-20160613', '2928454691_20180530_100910', '21874019371_20120330_082618_162', '5046728432_20190912_165528_888000', '21874019371_20120202_152524_800', '31008457529-20150422', '18953108760-20170509'}
{'26100447864-20151120', '3958863861-20140220', '26441976903-20180125', '13790385981-20170919', '37847697330-20160812', '7295991603-20161128', '8573381504-20170609', '41292354607-20150123', '12697747980-20160425', '8974639241-20150821', '12697747980-20150914', '3958863861-20160425', '26685639639-20150527', '1873918011-20170526', '12697747980-20140905', '26441976903-20180122', '29569544379-20140710', '29550499216-20151229', '12697747980-20161010', '24251918406-20041116', '37861465545-20161212'}
set()
5018 97 50


In [21]:
data_list = os.listdir('/nfs/masi/MCL/QA/maintain_list')
print (data_list)
print (sorted(data_list))

['20200815', 'before20191218', '20200901', 'code', '20200206_xinmeng', '20200130', 'before201906', '20200201', '20200306', '20200106', '20200816', '20200504', '20200902', '20200115', '20200311', '20200127', '20200120', '20200522', '20200131', '20200128', '20200512', 'readme.txt', '20200326', '20200110', '20200117', '20200207', '20200222', '20200121', '20200424', '20200909', '20200505']
['20200106', '20200110', '20200115', '20200117', '20200120', '20200121', '20200127', '20200128', '20200130', '20200131', '20200201', '20200206_xinmeng', '20200207', '20200222', '20200306', '20200311', '20200326', '20200424', '20200504', '20200505', '20200512', '20200522', '20200815', '20200816', '20200901', '20200902', '20200909', 'before201906', 'before20191218', 'code', 'readme.txt']


In [145]:
# temp 

f = open('/nfs/masi/MCL/QA/maintain_list/20200928/uptonow_uncertain.txt')
lines = f.readlines()
lines = [line.strip() for line in lines]
data = pd.DataFrame()

data['sess'] = lines
data.to_csv('/nfs/masi/MCL/QA/maintain_list/20200928/uptonow_uncertain.csv', index = False)



In [94]:

data_root = '/nfs/masi/MCL/xnat/MCL'
subj_list = os.listdir(data_root)
total_sess = []
f = open('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/need_sess_path.txt', 'w')
for subj in subj_list:
    sess_list = os.listdir(data_root + '/' + subj)
    for sess in sess_list:
        if sess in need_sess:
            f.write(data_root + '/' + subj + '/' + sess + '\n')
f.close()


In [92]:
print (len((set(total_sess)) & set(past_bad)))

241


In [93]:
need_sess = (set(total_sess)) & set(past_bad)
f = open('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/need_sess.csv', 'w')
for sess in need_sess:
    f.write(sess + '\n')
f.close()


In [103]:
df = pd.read_csv('/nfs/masi/MCL/xnat/xnat20201002_mightallbad/Riqiang_10_2_2020_16_37_59.csv')

all_sess = df['XNAT_CTSESSIONDATA ID'].tolist()

df1 = pd.read_csv('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/dicomQA_step1.csv')
df2 = pd.read_csv('/nfs/masi/MCL/xnat/xnat20200928_makeupjustin/dicomQA_step2.csv')

sess0928 = df1['sess'].tolist() + df2['sess'].tolist()

df_pre = pd.read_csv('/nfs/masi/MCL/QA/maintain_list/20200909/uptonow_good.csv')
sess_pre = df_pre['sess'].tolist()

In [104]:
need = set(all_sess) - set(sess0928) - set(sess_pre)

In [107]:
f = open('/nfs/masi/MCL/file/txt/for_download/xnat20201002_mightallbad.txt', 'w')
for sess in need:
    f.write(sess + '\n')
f.close()

In [124]:
df = pd.read_csv('/nfs/masi/MCL/QA/maintain_list/20200909/uptonow_bad.csv')
bad_list = df['sess'].tolist()
df = pd.read_csv('/nfs/masi/MCL/QA/maintain_list/20200909/uptonow_good.csv')
good_list = df['sess'].tolist()

In [125]:
print (len(bad_list), len(good_list))

329 4741


In [126]:
set(bad_list) & set(good_list)

set()